# 201_6.2_データ理解.ipynb

## 6.2 プラットフォーム全体で観測される報酬を最適化する方策学習

In [1]:
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import pandas as pd
from pandas import DataFrame
from tqdm import tqdm
from sklearn.utils import check_random_state
import seaborn as sns
import matplotlib.pyplot as plt
import japanize_matplotlib
plt.style.use('ggplot')

from dataset import generate_synthetic_data2
from policylearners import IPSBasedGradientPolicyLearner, CateBasedGradientPolicyLearner
from utils import softmax

### ログデータ(トレーニングデータ)のサイズ$n$を変化させたときの方策性能の変化

In [2]:
## シミュレーション設定
num_runs = 50 # シミュレーションの繰り返し回数
dim_x = 5 # 特徴量xの次元
num_actions = 10 # 行動数, |A|
beta = -0.1 # データ収集方策のパラメータ
max_iter = 21 # エポック数
test_data_size = 50000 # テストデータのサイズ
random_state = 12345
torch.manual_seed(random_state)
random_ = check_random_state(random_state)
num_actions_list = [2, 10] # 行動数
num_data_list = [100, 500, 1000, 2000] # トレーニングデータのサイズ

In [3]:
num_actions = num_actions_list[1]
## 期待報酬関数を定義するためのパラメータを抽出
theta_1 = random_.normal(size=(dim_x, num_actions))
M_1 = random_.normal(size=(dim_x, num_actions))
b_1 = random_.normal(size=(1, num_actions))
theta_0 = random_.normal(size=(dim_x, num_actions))
M_0 = random_.normal(size=(dim_x, num_actions))
b_0 = random_.normal(size=(1, num_actions))
## 学習された方策の真の性能を近似するためのテストデータを生成
test_data = generate_synthetic_data2(
    num_data=test_data_size, beta=beta,
    theta_1=theta_1, M_1=M_1, b_1=b_1, theta_0=theta_0, M_0=M_0, b_0=b_0,
    dim_context=dim_x, num_actions=num_actions, random_state = random_state
)
pi_0, q_x_a_1, q_x_a_0 = test_data["pi_0"], test_data["q_x_a_1"], test_data["q_x_a_0"]
pi_0_value = (pi_0 * q_x_a_1 + (1. - pi_0) * q_x_a_0).sum(1).mean()

In [4]:
num_data = num_data_list[0]
test_policy_value_list = []
_ = 0

## データ収集方策が形成する分布に従いログデータを生成
offline_logged_data = generate_synthetic_data2(
    num_data=num_data, beta=beta,
    theta_1=theta_1, M_1=M_1, b_1=b_1, theta_0=theta_0, M_0=M_0, b_0=b_0,
    dim_context=dim_x, num_actions=num_actions, random_state = _
)

true_value_of_learned_policies = dict()
true_value_of_learned_policies["logging"] = pi_0_value

## ログデータを用いてオフ方策学習を実行する
### 勾配ベースのアプローチ (IPS推定量で方策勾配を推定)
ips = IPSBasedGradientPolicyLearner(
    dim_x=dim_x, num_actions=num_actions, max_iter=max_iter, random_state=random_state + _
)
ips.fit(offline_logged_data, test_data)
pi_ips = ips.predict(test_data)
true_value_of_learned_policies["ips-pg"] = (pi_ips * q_x_a_1 + (1 - pi_ips) * q_x_a_0).sum(1).mean()
### 勾配ベースのアプローチ (新たに開発した推定量で方策勾配を推定)
cate = CateBasedGradientPolicyLearner(
    dim_x=dim_x, num_actions=num_actions, max_iter=max_iter, random_state=random_state + _
)
cate.fit(offline_logged_data, test_data)
pi_cate = cate.predict(test_data)
true_value_of_learned_policies["cate-pg"] = (pi_cate * q_x_a_1 + (1 - pi_cate) * q_x_a_0).sum(1).mean()

test_policy_value_list.append(true_value_of_learned_policies)

In [5]:
offline_logged_data.keys()

dict_keys(['num_data', 'num_actions', 'x', 'a', 'r', 'a_mat', 'r_mat', 'pi_0', 'pscore', 'pscore_mat', 'q_x_a_1', 'q_x_a_0', 'cate_x_a'])

In [6]:
offline_logged_data['x'][0:5] # 100 x 5

array([[ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799],
       [-0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ],
       [ 0.14404357,  1.45427351,  0.76103773,  0.12167502,  0.44386323],
       [ 0.33367433,  1.49407907, -0.20515826,  0.3130677 , -0.85409574],
       [-2.55298982,  0.6536186 ,  0.8644362 , -0.74216502,  2.26975462]])

In [7]:
offline_logged_data['a'][0:5] # 100,

array([6, 7, 6, 5, 4])

In [8]:
offline_logged_data['r'][0:5] # 100,

array([-2.71187456e+00, -1.16557267e-01,  1.83312752e-01, -1.48332513e-03,
       -1.51872727e+00])

In [9]:
offline_logged_data['a_mat'][0:5] # 100 x a

array([[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [10]:
offline_logged_data['r_mat'][0:5] # 100 x a

array([[-1.54803638e+00,  3.92523749e+00,  3.27264401e+00,
        -4.66325587e+00, -1.33671345e+00, -3.80441007e+00,
        -2.71187456e+00,  3.88213229e-01,  1.84347401e+00,
         7.21551246e-01],
       [ 1.92504899e+00,  2.37913232e+00,  1.52877231e-01,
        -3.35746062e+00,  2.91338278e-01,  9.66397293e-01,
         4.51964257e-01, -1.16557267e-01,  2.99750269e-02,
         4.23057505e-01],
       [-2.06563178e+00, -2.35068558e+00, -1.63887473e+00,
         6.78867776e-01,  2.12819107e-01,  3.11873405e-01,
         1.83312752e-01,  2.26058551e+00, -1.45847057e+00,
         1.90238572e+00],
       [-4.44226394e+00, -8.88543685e-01,  1.19997135e+00,
         5.42303119e-01, -2.09468581e+00, -1.48332513e-03,
         1.18063927e+00,  8.23596597e-01,  1.00595203e+00,
         9.55923666e-01],
       [-1.30241616e+00, -8.73720986e-02,  1.17579910e+00,
        -2.36083851e+00, -1.51872727e+00, -4.46327425e+00,
         1.29876877e+00, -7.68953555e-01,  1.93336425e+00,
         6.

In [11]:
offline_logged_data['pi_0'][0:5] # 100 x a

array([[0.08466615, 0.13932272, 0.09953897, 0.05385173, 0.08429327,
        0.07029619, 0.13718839, 0.11202252, 0.12244678, 0.09637328],
       [0.10093022, 0.10134481, 0.10397324, 0.09423109, 0.10118311,
        0.10196232, 0.09995899, 0.09876115, 0.09889369, 0.09876137],
       [0.09140668, 0.09474466, 0.09970321, 0.10462414, 0.09748314,
        0.10563132, 0.09647297, 0.10553351, 0.10284232, 0.10155806],
       [0.08415532, 0.0938042 , 0.09836095, 0.09954587, 0.09709193,
        0.10528356, 0.10194512, 0.1079261 , 0.10657685, 0.10531009],
       [0.08904502, 0.08935478, 0.09546663, 0.05650387, 0.16200582,
        0.07211176, 0.0931687 , 0.12429913, 0.10551753, 0.11252677]])

In [12]:
offline_logged_data['pscore'][0:5] # 100,

array([0.13718839, 0.09876115, 0.09647297, 0.10528356, 0.16200582])

In [13]:
offline_logged_data['pscore_mat'][0:5] # 100 x a

array([[0.91533385, 0.86067728, 0.90046103, 0.94614827, 0.91570673,
        0.92970381, 0.13718839, 0.88797748, 0.87755322, 0.90362672],
       [0.89906978, 0.89865519, 0.89602676, 0.90576891, 0.89881689,
        0.89803768, 0.90004101, 0.09876115, 0.90110631, 0.90123863],
       [0.90859332, 0.90525534, 0.90029679, 0.89537586, 0.90251686,
        0.89436868, 0.09647297, 0.89446649, 0.89715768, 0.89844194],
       [0.91584468, 0.9061958 , 0.90163905, 0.90045413, 0.90290807,
        0.10528356, 0.89805488, 0.8920739 , 0.89342315, 0.89468991],
       [0.91095498, 0.91064522, 0.90453337, 0.94349613, 0.16200582,
        0.92788824, 0.9068313 , 0.87570087, 0.89448247, 0.88747323]])

In [14]:
offline_logged_data['q_x_a_1'][0:5] # 100 x a

array([[ 6.10304973e-01,  1.51978243e+00,  3.09903716e+00,
         2.63684674e+00,  1.59594910e+00,  1.17791373e+00,
        -1.07924004e+00,  1.69721389e+00,  1.87409880e+00,
         6.66064968e-01],
       [ 8.99630794e-01,  8.33016950e-01,  1.05788887e+00,
        -8.32202060e-01,  7.96211737e-01,  5.58680754e-01,
        -8.01907314e-02, -1.16452942e+00, -1.80623313e-01,
        -1.23254094e+00],
       [-1.01790964e+00, -9.71158893e-01, -1.33248196e-03,
         1.04239847e+00,  1.52209484e-01,  5.79347050e-01,
        -7.53429712e-01,  2.36274977e-01,  5.57449514e-01,
         8.10329164e-01],
       [-5.28433340e-01, -7.01272085e-01,  8.95290628e-02,
         4.72129383e-01, -5.70712090e-01,  5.83947879e-01,
         1.19880613e+00,  1.92241240e-01,  5.21052931e-01,
         4.97581120e-01],
       [-2.20868782e-02,  2.58028375e+00,  2.21528113e+00,
         2.16303852e+00, -1.13408184e+00, -4.16379859e-01,
         1.26168334e+00, -5.14487242e-02,  5.48444953e-01,
         4.

In [83]:
offline_logged_data['q_x_a_0'][0:5] # 100 x a

array([[-1.93076881,  3.95947977,  2.17629717, -4.42904007, -0.9892628 ,
        -3.22314159,  1.20607914,  1.95598095,  3.02263195, -0.57987683],
       [ 1.02978872,  1.00416825,  1.48508889, -1.38883593,  0.9513946 ,
         0.79057834, -0.04672602, -1.2516221 , -0.25430464, -1.31961128],
       [-1.8430261 , -1.43760636,  0.04234349,  1.56783913, -0.02929885,
         1.20059366, -1.03910372,  0.84825781,  0.91111633,  1.03833342],
       [-2.20265988, -1.29004274, -0.02489921,  0.47744701, -0.81499663,
         1.14965307,  1.44228471,  1.00584138,  1.20884887,  1.06580644],
       [-1.5158962 ,  1.12120156,  1.41781893, -3.87909968,  3.35701068,
        -4.01943815,  0.22057147,  1.79023111,  0.75198564,  1.25652563]])

In [84]:
offline_logged_data['cate_x_a'][0:5] # 100 x a

array([[ 2.54107378e+00, -2.43969734e+00,  9.22739997e-01,
         7.06588681e+00,  2.58521190e+00,  4.40105532e+00,
        -2.28531918e+00, -2.58767060e-01, -1.14853314e+00,
         1.24594179e+00],
       [-1.30157925e-01, -1.71151301e-01, -4.27200014e-01,
         5.56633871e-01, -1.55182861e-01, -2.31897586e-01,
        -3.34647131e-02,  8.70926806e-02,  7.36813310e-02,
         8.70703343e-02],
       [ 8.25116463e-01,  4.66447468e-01, -4.36759706e-02,
        -5.25440668e-01,  1.81508338e-01, -6.21246612e-01,
         2.85674007e-01, -6.11982832e-01, -3.53666819e-01,
        -2.28004260e-01],
       [ 1.67422655e+00,  5.88770656e-01,  1.14428277e-01,
        -5.31762885e-03,  2.44284544e-01, -5.65705195e-01,
        -2.43478579e-01, -8.13600141e-01, -6.87795942e-01,
        -5.68225325e-01],
       [ 1.49380933e+00,  1.45908219e+00,  7.97462200e-01,
         6.04213819e+00, -4.49109253e+00,  3.60305830e+00,
         1.04111187e+00, -1.84167984e+00, -2.03540691e-01,
        -8.